<a href="https://colab.research.google.com/github/kevinsen-git/TF_Basics/blob/main/03_Modelling_Medical_Cost_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Modelling Medical Cost data
Lets do a regression anlaysis - from a Kaggle Dataset 

Dataset Description and Download 
https://www.kaggle.com/mirichoi0218/insurance/activity

A good Analysis based on R 
https://www.kaggle.com/goksunazlican/analysis-of-medical-cost-personal-datasets

A cracker based on Python - must read 
https://www.kaggle.com/asifpervezpolok/insurance-cost-prediction-88-1-r2-score

Lots of cool Graphics 
Lots of cool stats analysis 
Standard models 
Uses SikitLearn 

